In [1]:
import pandas as pd
import numpy as np

In [2]:
!git clone https://github.com/aviana-michaelj/nemesis-ml.git

Cloning into 'nemesis-ml'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 114 (delta 0), reused 2 (delta 0), pack-reused 109
Receiving objects: 100% (114/114), 2.26 MiB | 1.14 MiB/s, done.
Resolving deltas: 100% (34/34), done.


In [3]:
df = pd.read_csv('nemesis-ml/data/raw/Kaggle/train_sample.csv')
df=df.drop(columns = ['Unnamed: 0'])

In [5]:
df_type = pd.read_csv('nemesis-ml/data/raw/Kaggle/column_type.csv')

In [6]:
df.columns

Index(['Row_ID', 'Household_ID', 'Vehicle', 'Calendar_Year', 'Model_Year',
       'Blind_Make', 'Blind_Model', 'Blind_Submodel', 'Cat1', 'Cat2', 'Cat3',
       'Cat4', 'Cat5', 'Cat6', 'Cat7', 'Cat8', 'Cat9', 'Cat10', 'Cat11',
       'Cat12', 'OrdCat', 'Var1', 'Var2', 'Var3', 'Var4', 'Var5', 'Var6',
       'Var7', 'Var8', 'NVCat', 'NVVar1', 'NVVar2', 'NVVar3', 'NVVar4',
       'Claim_Amount'],
      dtype='object')

In [7]:
df_type

,Variable,Type
0,Row_ID,Delete
1,Household_ID,Nominal
2,Vehicle,Continuous
3,Calendar_Year,Nominal
4,Model_Year,Nominal
5,Blind_Make,Nominal
6,Blind_Model,Nominal
7,Blind_Submodel,Nominal
8,Cat1,Nominal
9,Cat2,Nominal


In [8]:
#function to get number of missing values in a column
def get_na_num(column): #input the whole column
    if column.dtype == np.object:
        return column.isnull().sum() + column[column == ''].count() + column[column == '?'].count()
    else:
        return column.isnull().sum()

In [9]:
#function to get number of valid values in a column
def get_valid_num(column): #input the whole column
    return len(column) - get_na_num(column)

In [10]:
#function to get minimum value in a column
def get_min(column): #input the whole column
    return min(column)

In [11]:
#function to get maximum value in a column
def get_max(column): #input the whole column
    return max(column)

In [12]:
#function to get mean in a column
def get_mean(column): #input the whole column
    return column.mean()

In [13]:
#function to get std in a column
def get_std(column): #input the whole column
    return column.std()

In [14]:
#function to get skewness in a column
def get_skew(column): #input the whole column
    return column.skew()

In [15]:
#function to get number of distict values in a column
def get_distinct_num(column): #input the whole column
    return len(column.unique().tolist())

In [16]:
#function to get count of each distict value in a column
def get_distinct_count(column): #input the whole column
    if get_distinct_num(column) > 5:
        print('Number of distict values is larger than 5. We stop updating the number of distinct values')
    else:
        return column.value_counts()

In [17]:
#function to get median in a column
import statistics
def get_median(column):
    if get_distinct_num(column) > 5:
        print('Number of distict values is larger than 5. We do not calculate median')
    else:
        return statistics.median(column)

In [18]:
#function to get mode and count for the mode in a column
def get_mode(column):
    return (column.mode()[0],column[column==column.mode()[0]].count())

In [19]:
#function to check whether a numerical column is continuous or discrete
def check_cont_or_dis(column):
    if get_distinct_num(column) > max(0.0001 * get_valid_num(column), 10):
        return ('continuous')
    else:
        return ('discrete')

In [20]:
# funtion to get column type
def column_type(column):
    return (df_type.loc[df_type['Variable'] == column, 'Type'].iloc[0])

In [21]:
#function to do basic variable screening and create basic statistical report
def Stats_Collection(df,df_type):
    for c in df:
        #exclude Target 
        if (column_type(c) != 'Flag_Continuous' and column_type(c) != 'Flag_Categorical'):
            print('Variable name: ',c)

            #Basic variable screening
            if get_na_num(df[c])/len(df[c]) > 0.5:
                print('More 50% missing values, drop this column\n')
                df = df.drop(columns=c)
                df_type = df_type.drop(index=int(df_type[df_type.Variable == c].index[0]))
                continue
            if (column_type(c) == 'Delete'):
                df = df.drop(columns=c)
                df_type = df_type.drop(index=int(df_type[df_type.Variable == c].index[0]))
                continue
            if (column_type(c) == 'Continuous') and (get_min(df[c]) == get_max(df[c])):
                print('All same value, drop this column\n')
                df = df.drop(columns=c)
                df_type = df_type.drop(index=int(df_type[df_type.Variable == c].index[0]))
                continue
            if (column_type(c) == 'Ordinal' or column_type(c) == 'Nominal') and (get_mode(df[c])[1]/get_valid_num(df[c]) > 0.95):
                print('Mode contains more than 95% cases, drop this column\n')
                df = df.drop(columns=c)
                df_type = df_type.drop(index=int(df_type[df_type.Variable == c].index[0]))
                continue
            if (column_type(c) == 'Nominal') and (get_distinct_num(df[c]) > 100):
                print('More than 100 categories, drop this column\n')
                df = df.drop(columns=c)
                df_type = df_type.drop(index=int(df_type[df_type.Variable == c].index[0]))
                continue

            #Basic statistic report
            print('Variable type: ', column_type(c))
            print ('Number of missing values: ',get_na_num(df[c]))
            print ('Number of valid values: ',get_valid_num(df[c]))
            if column_type(c) == 'Continuous' or column_type(c) == 'Ordinal':
                print('Minimum value: ', get_min(df[c]))
                print('Maximum value: ', get_max(df[c]))
            if column_type(c) == 'Continuous':
                print('Mean: ',get_mean(df[c]))
                print('Standard Deviation: ',get_std(df[c]))
                print('Skewness: ',get_skew(df[c]))
                print('Number of distinct values: ',get_distinct_num(df[c]))
                print('Number of cases for each distinct value: ')
                print(get_distinct_count(df[c]))
            else:
                print('Number of categories: ', get_distinct_num(df[c]))
                print('The counts of each category: ')
                print(get_distinct_count(df[c]))
                print('Mode: ', get_mode(df[c])[0],'Count: ',get_mode(df[c])[1])                
                print()
    return(df,df_type)

In [22]:
d = Stats_Collection(df,df_type)

Variable name:  Row_ID
Variable name:  Household_ID
More than 100 categories, drop this column

Variable name:  Vehicle
Variable type:  Continuous
Number of missing values:  0
Number of valid values:  5000
Minimum value:  1
Maximum value:  14
Mean:  2.2438
Standard Deviation:  1.4993369075136003
Skewness:  2.0299344354564077
Number of distinct values:  14
Number of cases for each distinct value: 
Number of distict values is larger than 5. We stop updating the number of distinct values
None
Variable name:  Calendar_Year
Variable type:  Nominal
Number of missing values:  0
Number of valid values:  5000
Number of categories:  2
The counts of each category: 
2005    4026
2006     974
Name: Calendar_Year, dtype: int64
Mode:  2005 Count:  4026

Variable name:  Model_Year
Variable type:  Nominal
Number of missing values:  0
Number of valid values:  5000
Number of categories:  27
The counts of each category: 
Number of distict values is larger than 5. We stop updating the number of distinct va

In [23]:
new_df_type = d[1]

In [24]:
new_df = d[0]

In [25]:
# function to identify outliers in continuous variables
# returns a bool panda series, a lower cutoff value and an upper cutoff value(used in outlier handling)
def outlier_identification(column):
    ori_mean = get_mean(column.dropna())
    ori_std = get_std(column.dropna())
    N_i = []
    X_i = []
    M_i = []
    for i in range(-3, 5):
        lower = ori_mean + (i - 1) * ori_std if i != -3 else -float('inf')
        upper = ori_mean + i * ori_std if i != 4 else float('inf')
        temp1 = column[(column <= upper) & (column > lower)]
        N_i.append(len(temp1))
        X_i.append(get_mean(temp1))
        M_i.append(np.var(temp1) * len(temp1))
    l = -3
    r = 4
    p = 0
    while 1:
        if N_i[l + 3] <= N_i[r + 3]:
            p_current = N_i[l + 3] / get_valid_num(column)
            if p + p_current < 0.05:
                l = l + 1
                p = p + p_current
            else:
                break
        else:
            p_current = N_i[r + 3] / get_valid_num(column)
            if p + p_current < 0.05:
                r = r - 1
                p = p + p_current
            else:
                break
    lower = ori_mean + (l - 1) * ori_std if l != -3 else -float('inf')
    upper = ori_mean + r * ori_std if r != 4 else float('inf')
    temp1 = column[(column <= upper) & (column > lower)]
    x_robust = get_mean(temp1)
    M_robust = 0
    N_sum = 0
    for i in range(l, r + 1):
        A_i = M_i[i + 3] + N_i[i + 3] * (x_robust - X_i[i + 3])**2
        M_robust = M_robust + A_i
        N_sum = N_sum + N_i[i + 3]
    sd_robust = np.sqrt(M_robust / (N_sum - 1))
    result = (column - x_robust < -3 * sd_robust) | (column + x_robust > 3 * sd_robust)
    lower_cutoff_value = x_robust - 3 * sd_robust
    upper_cutoff_value = x_robust + 3 * sd_robust
    return(result, lower_cutoff_value, upper_cutoff_value)
outlier_identification(new_df['Var1'])

(0       False
 1       False
 2       False
 3       False
 4       False
 5       False
 6       False
 7       False
 8       False
 9       False
 10      False
 11      False
 12      False
 13      False
 14      False
 15      False
 16      False
 17      False
 18      False
 19      False
 20      False
 21      False
 22      False
 23      False
 24      False
 25      False
 26      False
 27      False
 28      False
 29      False
         ...  
 4970    False
 4971    False
 4972    False
 4973    False
 4974    False
 4975    False
 4976    False
 4977    False
 4978    False
 4979    False
 4980    False
 4981    False
 4982    False
 4983    False
 4984    False
 4985    False
 4986    False
 4987    False
 4988     True
 4989     True
 4990    False
 4991    False
 4992    False
 4993    False
 4994    False
 4995    False
 4996    False
 4997     True
 4998    False
 4999    False
 Name: Var1, Length: 5000, dtype: bool, -2.8276437587410994, 3.10881280749214)

In [26]:
# function to trim outliers to cutoff values
def outlier_trim(df,column):
    (flag, lower, upper) = outlier_identification(column)
    df.loc[flag, column.name] = pd.Series(map(lambda x : (lower if x <= lower else upper), column[flag]))

In [27]:
# function to set outliers to missing values
def outlier_toNone(df,column):
    (flag, lower, upper) = outlier_identification(column)
    df.loc[flag, column.name] = None

In [28]:
print(sum(new_df['Var1'] > 3.1088128074921366))
print(sum(new_df['Var1'] < -2.827643758741096))
for c in new_df:
    if column_type(c) == 'Continuous':
        outlier_trim(new_df,new_df[c])
print(sum(new_df['Var1'] > 3.1088128074921366))
print(sum(new_df['Var1'] < -2.827643758741096))

124
0
0
0


In [29]:
# function to fill missing value and update statistic
def fill_missing_value(mydata, column_type):
    # filling missing value and updata statistic
    column_list = mydata.columns.values.tolist()
    typelist = list(column_type.iloc[:,1])
    i = 0
    for typ in typelist:
        column_data = mydata[column_list[i]].dropna()
        if (typ == 'Continuous'):
            mean_value = column_data.mean() # calculate mean
            mydata[column_list[i]] = mydata[column_list[i]].fillna(mean_value) # fill missing value with mean
            cont_sd = mydata[column_list[i]].std() # calculate standard deviation
            cont_skew = mydata[column_list[i]].skew() # calculate skewness
            print('')
            print('Column:', column_list[i])
            print('Column type: continuous')
            print('Mean:', mean_value)
            print('Standard deviation:', cont_sd)
            print('Skewness:', cont_skew)
            print('The number of missing values:', get_na_num(mydata[column_list[i]]))
            print('The number of valid values:', get_valid_num(mydata[column_list[i]]))
        elif (typ == 'Ordinal' and mydata[column_list[i]].dtype != 'object'):
            num_median = column_data.median() # calculate median 
            mydata[column_list[i]] = mydata[column_list[i]].fillna(num_median) # fill missing value with median
            count_median_num = mydata[column_list[i]][mydata[column_list[i]] == num_median].count() # count the the number of cases in the median category
            print('')
            print('Column:', column_list[i])
            print('Column type: num_ordinal')
            print('Median:', num_median)
            print('The number of cases in the median category:', count_median_num)
            print('The number of missing values:', get_na_num(mydata[column_list[i]]))
            print('The number of valid values:', get_valid_num(mydata[column_list[i]]))
        elif (typ == 'Ordinal' and mydata[column_list[i]].dtype == 'object'):
            mode_value = column_data.mode()[0] # calculate mode
            mydata[column_list[i]] = mydata[column_list[i]].fillna(mode_value) # fill missing valye with mode
            count_mode = mydata[column_list[i]][mydata[column_list[i]] == mode_value].count() # count the the number of cases in the modal category
            print('')
            print('Column:', column_list[i])
            print('Column type: cat_ordinal')
            print('Mode:', mode_value)
            print('The number of cases in the modal category:', count_mode)
            print('The number of missing values:', get_na_num(mydata[column_list[i]]))
            print('The number of valid values:', get_valid_num(mydata[column_list[i]]))
        else:
            mode_value = column_data.mode()[0] # calculate mode
            mydata[column_list[i]] = mydata[column_list[i]].fillna(mode_value) # fill missing valye with mode
            count_mode = mydata[column_list[i]][mydata[column_list[i]] == mode_value].count() # count the the number of cases in the modal category
            print('')
            print('Column:', column_list[i])
            print('Column type: nominal')
            print('Mode:', mode_value)
            print('The number of cases in the modal category:', count_mode)
            print('The number of missing values:', get_na_num(mydata[column_list[i]]))
            print('The number of valid values:', get_valid_num(mydata[column_list[i]]))
        i = i + 1
    # add column type at the last row
    print('add column type at the last row:')
    return (mydata)

In [30]:
new_df = new_df.replace('?',np.NaN)
fill_missing_value(new_df,new_df_type)


Column: Vehicle
Column type: continuous
Mean: 3.183613197220352
Standard deviation: 1.9279074114234616
Skewness: 0.022769597720921908
The number of missing values: 0
The number of valid values: 5000

Column: Calendar_Year
Column type: nominal
Mode: 2005
The number of cases in the modal category: 4026
The number of missing values: 0
The number of valid values: 5000

Column: Model_Year
Column type: nominal
Mode: 2002
The number of cases in the modal category: 506
The number of missing values: 0
The number of valid values: 5000

Column: Blind_Make
Column type: nominal
Mode: AJ
The number of cases in the modal category: 858
The number of missing values: 0
The number of valid values: 5000

Column: Cat1
Column type: nominal
Mode: D
The number of cases in the modal category: 1352
The number of missing values: 0
The number of valid values: 5000

Column: Cat2
Column type: nominal
Mode: C
The number of cases in the modal category: 4138
The number of missing values: 0
The number of valid values:

,Vehicle,Calendar_Year,Model_Year,Blind_Make,Cat1,Cat2,Cat3,Cat6,Cat8,Cat9,...,Var5,Var6,Var7,Var8,NVCat,NVVar1,NVVar2,NVVar3,NVVar4,Claim_Amount
0,5.411107,2005,2005,K,D,C,F,C,C,A,...,1.008912,0.261040,0.907793,-0.077998,M,-0.231530,-0.266117,-0.272337,-0.251419,0.0
1,5.411107,2005,2003,Q,B,C,A,E,A,B,...,1.240851,0.432987,-0.726459,0.204785,O,-0.231530,-0.266117,-0.272337,-0.251419,0.0
2,1.000000,2005,1998,AR,B,C,A,C,A,B,...,-0.971487,-1.405797,-0.837048,-1.176858,F,-0.231530,-0.266117,-0.272337,-0.251419,0.0
3,1.000000,2006,1998,AR,B,C,A,C,A,B,...,-0.971487,-1.405797,-0.837048,-1.176858,F,-0.231530,-0.266117,-0.272337,-0.251419,0.0
4,5.411107,2005,2001,D,J,C,B,D,A,B,...,0.812656,2.112691,1.534462,2.347260,F,-0.231530,-0.266117,-0.272337,-0.251419,0.0
5,5.411107,2006,2001,D,J,C,B,D,A,B,...,0.812656,2.112691,1.534462,2.347260,F,-0.231530,-0.266117,-0.272337,-0.251419,0.0
6,1.000000,2006,2001,AJ,G,C,A,E,A,B,...,0.580718,0.551128,0.416289,-0.024395,M,-0.231530,-0.266117,-0.272337,-0.251419,0.0
7,5.411107,2006,2002,AQ,B,C,B,D,B,A,...,0.527193,-0.023200,-0.701884,0.226663,M,-0.231530,-0.266117,-0.272337,-0.251419,0.0
8,5.411107,2005,2002,AQ,B,C,B,D,B,A,...,0.527193,-0.023200,-0.701884,0.226663,M,-0.231530,-0.266117,-0.272337,-0.251419,0.0
9,1.000000,2005,1995,BW,D,C,E,D,A,B,...,0.176312,0.283264,0.969232,-0.792339,N,2.054683,-0.266117,-0.272337,-0.251419,0.0


In [44]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(df_variable)
X_pca=pca.transform(df_variable) 
PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
svd_solver='auto', tol=0.0, whiten=False)
print(pca.explained_variance_ratio_)  

[0.68662931 0.16399365]


In [48]:
X_pca = pd.DataFrame(X_pca)

In [49]:
X_pca

,0,1
0,-6.009852,0.522930
1,-3.927077,0.226881
2,1.972014,-3.424450
3,1.964676,-3.422812
4,-2.714973,3.364158
5,-2.722312,3.365796
6,-1.931669,0.578892
7,-2.759945,-0.723206
8,-2.752607,-0.724843
9,4.183368,1.005155
